In [1]:
from unsloth import FastLanguageModel
from trl import SFTTrainer
import torch
from transformers import TrainingArguments, TextStreamer
from datasets import DatasetDict, load_dataset
from rich import print
from rich.progress import track
import evaluate

2024-05-31 23:40:31.520940: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-31 23:40:31.550252: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-31 23:40:31.994495: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
dataset = DatasetDict.load_from_disk('../data/instructions')
dataset_train = dataset["train"]
dataset_test = dataset["test"]

In [4]:
# convert huggingface dataset to df
df = dataset_test.to_pandas()
df

,instruction,response,vietnamese,english,category,metadata
0,"Cho cặp câu tiếng Việt và tiếng Anh sau, hãy c...",Đúng,Mày làm ăn như vậy thì ai mà chịu nổi,Who can stand you if you keep doing like that,classification,generated
1,"Cho cặp câu tiếng Việt và tiếng Anh sau, hãy c...",Đúng,"Mày làm ơn đi, tao chán mày lắm rồi","Please, I'm so fed up with you",classification,generated
2,"Cho cặp câu tiếng Việt và tiếng Anh sau, hãy c...",Sai,Bữa nay tao chả muốn nói chuyện với mày,I don't feel like talking to you today,classification,generated
3,"Cho cặp câu tiếng Việt và tiếng Anh sau, hãy c...",Sai,Tao chả hỉu mày đang nói gì hết trơn,I have no idea what you're talking about,classification,generated
4,Dịch câu sau từ tiếng Việt sang tiếng Anh: \nT...,Do you remember when I saved you from them?,Mày có nhớ lúc tao cứu mày khỏi bọn nó không?,Do you remember when I saved you from them?,translation,generated
...,...,...,...,...,...,...
1638,"Cho cặp câu tiếng Việt và tiếng Anh sau, hãy c...",Đúng,và cô học lên cấp 3,she went to high school,classification,nonteencode
1639,"Cho cặp câu tiếng Việt và tiếng Anh sau, hãy c...",Đúng,chúng tôi phát bao cao su cho mọi người trên đ...,and then we gave condoms out everywhere on the...,classification,nonteencode
1640,Sử dụng các gợi ý căn chỉnh sau và dịch từ tiế...,we'd like to spend time with him he looks fri...,chúng ta thích đi chơi cùng anh ta anh ta nhì...,we'd like to spend time with him he looks fri...,translation_hint,nonteencode
1641,Dịch câu sau từ tiếng Việt sang tiếng Anh: \nT...,so while you're sitting around with the other ...,khi tôi ngồi với các bệnh nhân khác trong bộ p...,so while you're sitting around with the other ...,translation,nonteencode


In [5]:
df_teencode = df[df['metadata'] == 'teencode']
df_teencode

,instruction,response,vietnamese,english,category,metadata
1146,"Cho cặp câu tiếng Việt và tiếng Anh sau, hãy c...",Đúng,"Đôi khi cũng muốn dỗi lắm, nhưng nghĩ lại đéo ...","Sometimes I really want to sulk, but then I re...",classification,teencode
1147,Sử dụng các gợi ý căn chỉnh sau và dịch từ tiế...,Doing business in a sloppy or messy way.,làm ăn nhố nhăng,Doing business in a sloppy or messy way.,translation_hint,teencode
1148,Dịch câu sau từ tiếng Việt sang tiếng Anh: \nT...,That is such a good response.,phản dame gắt vãi,That is such a good response.,translation,teencode
1149,Dịch câu sau từ tiếng Việt sang tiếng Anh: \nT...,"So, you're saying if I see someone getting too...",Vậy thấy đứa nào thân thân với mình là mình bl...,"So, you're saying if I see someone getting too...",translation,teencode
1150,"Cho cặp câu tiếng Việt và tiếng Anh sau, hãy c...",Đúng,Vậy thấy đứa nào thân thân với mình là mình bl...,"So, you're saying if I see someone getting too...",classification,teencode
...,...,...,...,...,...,...
1565,"Cho cặp câu tiếng Việt và tiếng Anh sau, hãy c...",Đúng,tao buồn mấy năm nay rồi,I've been sad for years now.,classification,teencode
1566,"Cho cặp câu tiếng Việt và tiếng Anh sau, hãy c...",Đúng,động lực để phụ nữ chiến đấu chính là gia đình...,The motivation for women to fight is their fam...,classification,teencode
1567,Dịch câu sau từ tiếng Việt sang tiếng Anh: \nT...,I'm just worried the state won't have enough r...,mink chỉ lo là nhà nước không đủ gạo nuôi mấy anh,I'm just worried the state won't have enough r...,translation,teencode
1568,"Cho cặp câu tiếng Việt và tiếng Anh sau, hãy c...",Đúng,mink chỉ lo là nhà nước không đủ gạo nuôi mấy anh,I'm just worried the state won't have enough r...,classification,teencode


In [6]:
df_teencode = df_teencode.drop_duplicates(subset=['vietnamese'], keep='first')
df_teencode

,instruction,response,vietnamese,english,category,metadata
1146,"Cho cặp câu tiếng Việt và tiếng Anh sau, hãy c...",Đúng,"Đôi khi cũng muốn dỗi lắm, nhưng nghĩ lại đéo ...","Sometimes I really want to sulk, but then I re...",classification,teencode
1147,Sử dụng các gợi ý căn chỉnh sau và dịch từ tiế...,Doing business in a sloppy or messy way.,làm ăn nhố nhăng,Doing business in a sloppy or messy way.,translation_hint,teencode
1148,Dịch câu sau từ tiếng Việt sang tiếng Anh: \nT...,That is such a good response.,phản dame gắt vãi,That is such a good response.,translation,teencode
1149,Dịch câu sau từ tiếng Việt sang tiếng Anh: \nT...,"So, you're saying if I see someone getting too...",Vậy thấy đứa nào thân thân với mình là mình bl...,"So, you're saying if I see someone getting too...",translation,teencode
1151,Sử dụng các gợi ý căn chỉnh sau và dịch từ tiế...,It's best not to have a crush on anyone but yo...,tốt nhất là đừng crush ai ngoài bản thân ra cả...,It's best not to have a crush on anyone but yo...,translation_hint,teencode
...,...,...,...,...,...,...
1564,Dịch câu sau từ tiếng Việt sang tiếng Anh: \nT...,Laughing like a laughing stock.,cười như cục cức trôi sông lạc chợ,Laughing like a laughing stock.,translation,teencode
1565,"Cho cặp câu tiếng Việt và tiếng Anh sau, hãy c...",Đúng,tao buồn mấy năm nay rồi,I've been sad for years now.,classification,teencode
1566,"Cho cặp câu tiếng Việt và tiếng Anh sau, hãy c...",Đúng,động lực để phụ nữ chiến đấu chính là gia đình...,The motivation for women to fight is their fam...,classification,teencode
1567,Dịch câu sau từ tiếng Việt sang tiếng Anh: \nT...,I'm just worried the state won't have enough r...,mink chỉ lo là nhà nước không đủ gạo nuôi mấy anh,I'm just worried the state won't have enough r...,translation,teencode


# Base

In [8]:
max_seq_length = 4096  # Supports RoPE Scaling interally, so choose any!
# get instruction dataset from ../data/instructions

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="../models/vinallama-7b/checkpoint-2961",
    max_seq_length=max_seq_length,
    dtype=None,
    load_in_4bit=True,
)

FastLanguageModel.for_inference(model)

/home/nguyen/miniconda3/envs/nlp/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


==((====))==  Unsloth: Fast Llama patching release 2024.4
   \\   /|    GPU: NVIDIA GeForce RTX 3070 Ti. Max memory: 8.0 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Unsloth: ../models/vinallama-7b/checkpoint-2961 has no tokenizer.model file.
Just informing you about this - this is not a critical error.


Unsloth 2024.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [5]:
def format_prompt_func(example):
    if isinstance(example['instruction'], str):
        return f"### Instruction:\n{example['instruction']}\n\n### Response:\n{example['response']}"
    
    output_texts = []
    for i in range(len(example['instruction'])):
        text = f"### Instruction:\n{example['instruction'][i]}\n\n### Response:\n{example['response'][i]}"
        output_texts.append(text)
    return output_texts

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    formatting_func=format_prompt_func,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args = TrainingArguments(
        output_dir="../models/vinallama-7b/eval",
        fp16_full_eval = True,
        per_device_eval_batch_size = 1,
        eval_accumulation_steps = 4,
        evaluation_strategy = "steps",
        eval_steps = 1,
    ),
)

/home/nguyen/miniconda3/envs/nlp/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [3]:
benchmark_dataset = load_dataset("csv", data_files={"benchmark": "../data/VLSP2020_benchmark_sampled.csv"})
num_examples = len(benchmark_dataset["benchmark"])
predictions = []
references = benchmark_dataset["benchmark"]['english'][:num_examples]

Generating benchmark split: 0 examples [00:00, ? examples/s]

In [12]:
# text_streamer = TextStreamer(tokenizer)
predictions = []
def format_inference_prompt(example):
    return f"""
### Instruction:
Dịch câu sau từ tiếng Việt sang tiếng Anh: 
Tiếng Việt: {example}
Tiếng Anh:

### Response:
"""

def generate_predictions(model, tokenizer, df, num_examples, save_path):
    for example in track(df['vietnamese'].iloc[:num_examples], total=num_examples):
        inputs = tokenizer(
            [
                format_inference_prompt(example),
            ],
            return_tensors="pt",
        ).to("cuda")

        outputs = model.generate(
            **inputs, max_new_tokens=128
        )
        
        response = tokenizer.batch_decode(outputs)
        response_split = response[0].split("\n")
        
        response_token_index = response_split.index("### Response:")
        translation = response_split[response_token_index + 1]

        predictions.append(translation)

    with open(save_path, "w", encoding='utf8') as f:
        for prediction in predictions:
            f.write(prediction + "\n")    


In [13]:
num_examples = len(df_teencode)
path = "../data/predictions_vinallama_7b_teencode.txt"
generate_predictions(model, tokenizer, df_teencode, num_examples, path)

Output()

# Chat

In [6]:
max_seq_length = 4096
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="../models/vinallama-7b-chat/checkpoint-2961",
    max_seq_length=max_seq_length,
    dtype=None,
    load_in_4bit=True,
)
FastLanguageModel.for_inference(model)

/home/nguyen/miniconda3/envs/nlp/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


==((====))==  Unsloth: Fast Llama patching release 2024.4
   \\   /|    GPU: NVIDIA GeForce RTX 3070 Ti. Max memory: 8.0 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Unsloth: ../models/vinallama-7b-chat/checkpoint-2961 has no tokenizer.model file.
Just informing you about this - this is not a critical error.


Unsloth 2024.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [7]:
text_streamer = TextStreamer(tokenizer)

def format_inference_prompt(example):
    return f"""Dịch câu sau từ tiếng Việt sang tiếng Anh: 
Tiếng Việt: {example}
Tiếng Anh:
"""

predictions = []

def generate_predictions_chat(model, tokenizer, df, num_examples, save_path):
    for example in track(df['vietnamese'].iloc[:num_examples], total=num_examples):
        messages = [
            {"role": "system", "content": "Bạn là một trợ lí AI hữu ích trong việc dịch thuật tiếng Việt sang Tiếng Anh. Hãy trả lời người dùng một cách chính xác."},
            {"role": "user", "content": format_inference_prompt(example)},
        ]
        
        inputs = tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt",
        ).to("cuda")
        

        outputs = model.generate(
            inputs, max_new_tokens=128
        )
        
        response = tokenizer.batch_decode(outputs[:, inputs.shape[1]:])[0]
        
        if "<|im_end|>" in response:
            translation = response.split("<|im_end|>")[0].strip()
        else:
            translation = response.strip()
            
        
        # response_token_index = response_split.index("### Response:")
        # translation = response_split[response_token_index + 1]

        predictions.append(translation)

    with open(save_path, "w", encoding='utf8') as f:
        for prediction in predictions:
            f.write(prediction + "\n")    


In [8]:
# benchmark_dataset = load_dataset("csv", data_files={"benchmark": "../data/VLSP2020_benchmark_sampled.csv"})
num_examples = len(df_teencode)
predictions = []

generate_predictions_chat(model, tokenizer, df_teencode, num_examples, "../data/predictions_vinallama_7b_chat_teencode.txt")

Output()

# Evaluate

In [12]:
with open("../data/predictions_vinallama_7b_teencode.txt", "r", encoding='utf8') as f:
    predictions_vinallama_7b = f.readlines()

with open("../data/predictions_vinallama_7b_chat_teencode.txt", "r", encoding='utf8') as f:
    predictions_vinallama_7b_chat = f.readlines()

assert len(predictions_vinallama_7b) == len(predictions_vinallama_7b_chat)
num_examples = len(predictions_vinallama_7b)
references = df_teencode['english'].tolist()
sources = df_teencode['vietnamese'].tolist()

## BLEU

In [13]:
bleu = evaluate.load("bleu")

results_vinallama_7b = bleu.compute(predictions=predictions_vinallama_7b, references=references)
results_vinallama_7b_chat = bleu.compute(predictions=predictions_vinallama_7b_chat, references=references)

for key, value in results_vinallama_7b.items():
    print(f"Vinallama 7B {key}: {value}")
    print(f"Vinallama 7B Chat {key}: {results_vinallama_7b_chat[key]}")

Vinallama 7B bleu: 0.6941782058664238

Vinallama 7B Chat bleu: 0.7364245484039528

Vinallama 7B precisions: [0.7768510984540277, 0.7058187324248324, 0.666512702078522, 0.6353960396039604]

Vinallama 7B Chat precisions: [0.8244996701121619, 0.7529384109073813, 0.7140333165068148, 0.6824489795918367]

Vinallama 7B brevity_penalty: 1.0

...

Vinallama 7B Chat length_ratio: 0.9930115745796025

Vinallama 7B translation_length: 4916

Vinallama 7B Chat translation_length: 4547

Vinallama 7B reference_length: 4579

Vinallama 7B Chat reference_length: 4579

## BERTSCORE

In [14]:
bertscore = evaluate.load("bertscore")

results_vinallama_7b = bertscore.compute(predictions=predictions_vinallama_7b, references=references, lang="en")
results_vinallama_7b_chat = bertscore.compute(predictions=predictions_vinallama_7b_chat, references=references, lang="en")

for key, value in results_vinallama_7b.items():
    
    if isinstance(value, list):
        vinallama_7b_tensor = torch.tensor(value, dtype=torch.float32)
        vinallama_7b_chat_tensor = torch.tensor(results_vinallama_7b_chat[key], dtype=torch.float32)
        
        avg_vinallama_7b = torch.mean(vinallama_7b_tensor).item()
        avg_vinallama_7b_chat = torch.mean(vinallama_7b_chat_tensor).item()
        print(f"Vinallama 7B {key}: {avg_vinallama_7b}")
        print(f"Vinallama 7B Chat {key}: {avg_vinallama_7b_chat}")
    else:
        print(f"Vinallama 7B {key}: {value}")
        print(f"Vinallama 7B Chat {key}: {results_vinallama_7b_chat[key]}")

/home/nguyen/miniconda3/envs/nlp/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Vinallama 7B precision: 0.9627518653869629

Vinallama 7B Chat precision: 0.9665407538414001

Vinallama 7B recall: 0.9661794304847717

Vinallama 7B Chat recall: 0.9666844010353088

Vinallama 7B f1: 0.9643222689628601

Vinallama 7B Chat f1: 0.9665389657020569

Vinallama 7B hashcode: roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.40.2)

Vinallama 7B Chat hashcode: roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.40.2)

## BLEURT

In [15]:
bleurt = evaluate.load("bleurt", module_type="metric", checkpoint="bleurt-large-512")

results_vinallama_7b = bleurt.compute(predictions=predictions_vinallama_7b, references=references)
results_vinallama_7b_chat = bleurt.compute(predictions=predictions_vinallama_7b_chat, references=references)

for key, value in results_vinallama_7b.items():
    
    vinallama_7b_tensor = torch.tensor(value, dtype=torch.float32)
    vinallama_7b_chat_tensor = torch.tensor(results_vinallama_7b_chat[key], dtype=torch.float32)
    
    avg_vinallama_7b = torch.mean(vinallama_7b_tensor).item()
    avg_vinallama_7b_chat = torch.mean(vinallama_7b_chat_tensor).item()
    
    print(f"Vinallama 7B {key}: {avg_vinallama_7b}")
    print(f"Vinallama 7B Chat {key}: {avg_vinallama_7b_chat}")

Using default BLEURT-Base checkpoint for sequence maximum length 128. You can use a bigger model for better results with e.g.: evaluate.load('bleurt', 'bleurt-large-512').


INFO:tensorflow:Reading checkpoint /home/nguyen/.cache/huggingface/metrics/bleurt/default/downloads/extracted/49ae870ec41970df36d4cb594bb8991ef69b9fe584090a8c05e6b35a23eb644e/bleurt-base-128.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint bert_custom
INFO:tensorflow:Loads full paths and checks that files exists.
INFO:tensorflow:... name:bert_custom
INFO:tensorflow:... vocab_file:vocab.txt
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... do_lower_case:True
INFO:tensorflow:... max_seq_length:128
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Creating WordPiece tokenizer.
INFO:tensorflow:WordPiece tokenizer instantiated.
INFO:tensorflow:Creating Eager Mode predictor.
INFO:tensorflow:Loading model.


2024-05-31 23:42:50.426101: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-31 23:42:50.434166: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-31 23:42:50.434186: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-31 23:42:50.437790: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-31 23:42:50.437816: I external/local_xla/xla/stream_executor

INFO:tensorflow:BLEURT initialized.


INFO:tensorflow:BLEURT initialized.


Vinallama 7B scores: 0.4170148968696594

Vinallama 7B Chat scores: 0.4624171853065491

## MANUAL

In [16]:
num_examples = 20

# generate random indices
# create seed for reproducibility
torch.manual_seed(42)
indices = torch.randperm(len(predictions_vinallama_7b))[:num_examples].tolist()

predictions_vinallama_7b_sampled = [predictions_vinallama_7b[i] for i in indices]
predictions_vinallama_7b_chat_sampled = [predictions_vinallama_7b_chat[i] for i in indices]

references_sampled = [references[i] for i in indices]
sources_sampled = [sources[i] for i in indices]

for prediction_vinallama_7b, prediction_vinallama_7b_chat, reference, source in zip(predictions_vinallama_7b_sampled, predictions_vinallama_7b_chat_sampled, references_sampled, sources_sampled):
    print(f"Vietnamese: {source}")
    print(f"Reference: {reference}")
    print(f"Vinallama 7B: {prediction_vinallama_7b}")
    print(f"Vinallama 7B Chat: {prediction_vinallama_7b_chat}")
    print("\n")

Vietnamese: per sao bố lại đủ bình tĩnh để xem hết cái trailer này nhỉ . sợ vãi

Reference: Per, why did Dad have enough composure to watch this entire trailer? It's scary as fuck.

Vinallama 7B: Per, why did Dad have enough composure to watch this entire trailer? It's scary as fuck.

Vinallama 7B Chat: Per, why did Dad have enough composure to watch this entire trailer? It's scary as fuck.

Vietnamese: chuẩn bài

...

Vinallama 7B Chat: The story's turned out like a dramatic novel... you guys keep bullying him and you'll regret it

Vietnamese: con gái chả thế, con trai đi, dume mày có nhận hàng không

Reference: Girls no way, boys go, bro do you accept the stuff?

Vinallama 7B: Girls no way, boys go, bro do you accept the stuff?

Vinallama 7B Chat: Damn, it's the guys who go, bro if you wanna get stuff, I'm ready to deliver it